In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.width', 1400)

In [ ]:
def str_to_list(s):
    for c in "'[]":
        s = s.replace(c, '')
    return s.split(', ')

In [42]:
# 
df_recipes = pd.read_csv('dataset/RAW_recipes.csv') # 
df_interact = pd.read_csv('dataset/RAW_interactions.csv') # ratings

In [ ]:
print(df_recipes.head())
print(df_interact.head())
print(len(df_recipes))
print(len(df_interact))

#### **1. Initial Data Exploration:** Explore the distribution of recipes based on key features such as minutes, tags, n_steps, and n_ingredients, and visualize the distribution of recipes for each of these features. 

In [54]:
# 
df_recipes['ntags'] = df_recipes['tags'].apply( lambda tags: len(str_to_list(tags)) )
df_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,ntags
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,20
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,20
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,9
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,30
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,21


In [52]:

a = []
for i in range(11):
    # print(n_tags[i])
    l = str_to_list(df_recipes['ingredients'][i])
    print(len(l))
    for x in l:
        print(x)
    print()
    # a.append( l )

7
winter squash
mexican seasoning
mixed spice
honey
butter
olive oil
salt

6
prepared pizza crust
sausage patty
eggs
milk
salt and pepper
cheese

13
ground beef
yellow onions
diced tomatoes
tomato paste
tomato soup
rotel tomatoes
kidney beans
water
chili powder
ground cumin
salt
lettuce
cheddar cheese

11
spreadable cheese with garlic and herbs
new potatoes
shallots
parsley
tarragon
olive oil
red wine vinegar
salt
pepper
red bell pepper
yellow bell pepper

8
tomato juice
apple cider vinegar
sugar
salt
pepper
clove oil
cinnamon oil
dry mustard

4
milk
vanilla ice cream
frozen apple juice concentrate
apple

9
fennel seeds
green olives
ripe olives
garlic
peppercorn
orange rind
orange juice
red chile
extra virgin olive oil

23
pork spareribs
soy sauce
fresh garlic
fresh ginger
chili powder
fresh coarse ground black pepper
salt
fresh cilantro leaves
tomato sauce
brown sugar
yellow onion
white vinegar
honey
a.1. original sauce
liquid smoke
cracked black pepper
cumin
dry mustard
cinnamon stic

In [34]:
a

0    [[, ', 6, 0, -, m, i, n, u, t, e, s, -, o, r, ...
1    [[, ', 3, 0, -, m, i, n, u, t, e, s, -, o, r, ...
2    [[, ', t, i, m, e, -, t, o, -, m, a, k, e, ', ...
3    [[, ', 6, 0, -, m, i, n, u, t, e, s, -, o, r, ...
4    [[, ', w, e, e, k, n, i, g, h, t, ', ,,  , ', ...
5    [[, ', 1, 5, -, m, i, n, u, t, e, s, -, o, r, ...
6    [[, ', 1, 5, -, m, i, n, u, t, e, s, -, o, r, ...
7    [[, ', w, e, e, k, n, i, g, h, t, ', ,,  , ', ...
8    [[, ', w, e, e, k, n, i, g, h, t, ', ,,  , ', ...
9    [[, ', w, e, e, k, n, i, g, h, t, ', ,,  , ', ...
Name: tags, dtype: object